The following block copies much of the code from the previous notebooks - it sets up our notebook with our data that has been "cleaned" and with its new features. See the previous parts for explanations of each step.

In [1]:
#setup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

train_df = pd.read_csv('train.csv')  
test_df = pd.read_csv('test.csv')
#combine = [train_df, test_df]
test_df_predictions = pd.read_csv('test.csv')

train_df.drop('Cabin', 1, inplace=True)
test_df.drop('Cabin', 1, inplace=True)
train_df.drop('Ticket', 1, inplace=True)
test_df.drop('Ticket', 1, inplace=True)
train_df.drop('PassengerId', 1, inplace=True)
test_df.drop('PassengerId', 1, inplace=True)
train_df.drop('Embarked', 1, inplace=True)
test_df.drop('Embarked', 1, inplace=True)

train_df['Age'].fillna(train_df['Age'].median(),inplace=True)
test_df['Age'].fillna(test_df['Age'].median(),inplace=True)

test_df['Fare'].fillna(test_df['Fare'].median(),inplace=True)

df_sex = pd.get_dummies(train_df[['Sex']])
train_new = pd.concat([train_df,df_sex],axis=1)
train_new.drop('Sex',1,inplace=True)
df_sex_test = pd.get_dummies(test_df[['Sex']])
test_new = pd.concat([test_df,df_sex_test],axis=1)
test_new.drop('Sex',1,inplace=True)

def extract_title():
    train_new['Title'] = train_new['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    test_new['Title'] = test_new['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

extract_title()
train_new.drop('Name',1,inplace=True)
test_new.drop('Name',1,inplace=True)

def extractMrs():
    married_female = []
    for i in train_new['Title']:
        if i == "Mrs":
            married_female.append(1)
        else:
            married_female.append(0)
    train_new['married_female'] = married_female
    
    married_female_test = []
    for i in test_new['Title']:
        if i == "Mrs":
            married_female_test.append(1)
        else:
            married_female_test.append(0)
    test_new['married_female_test'] = married_female_test
    
    
extractMrs()
train_new.drop('Title',1,inplace=True)
test_new.drop('Title',1,inplace=True)

train_new["FamilySize"] = train_new['Parch'] + train_new['SibSp'] + 1
test_new["FamilySize"] = test_new['Parch'] + test_new['SibSp'] + 1

def travelAlone():
    travelAlonePassenger = []
    for i in train_new["FamilySize"]:
        if i == 1:
            travelAlonePassenger.append(1)
        else:
            travelAlonePassenger.append(0)
    train_new["travelAlonePassenger"] = travelAlonePassenger
    
    travelAlonePassenger_test = []
    for i in test_new["FamilySize"]:
        if i == 1:
            travelAlonePassenger_test.append(1)
        else:
            travelAlonePassenger_test.append(0)
    test_new["travelAlonePassenger"] = travelAlonePassenger_test

def bigFamily():
    bigFamilyPassenger = []
    for i in train_new["FamilySize"]:
        if i > 3:
            bigFamilyPassenger.append(1)
        else:
            bigFamilyPassenger.append(0)
    train_new["bigFamilyPassenger"] = bigFamilyPassenger
    
    bigFamilyPassenger_test = []
    for i in test_new["FamilySize"]:
        if i > 3:
            bigFamilyPassenger_test.append(1)
        else:
            bigFamilyPassenger_test.append(0)
    test_new["bigFamilyPassenger"] = bigFamilyPassenger_test
    
travelAlone()
bigFamily()

def ticketFareThreshold():
    ticketFareThreshold = []
    for i in train_new["Fare"]:
        if i > 20:
            ticketFareThreshold.append(1)
        else:
            ticketFareThreshold.append(0)
    train_new["ticketFareThreshold"] = ticketFareThreshold
    
    ticketFareThreshold_test = []
    for i in test_new["Fare"]:
        if i > 20:
            ticketFareThreshold_test.append(1)
        else:
            ticketFareThreshold_test.append(0)
    test_new["ticketFareThreshold"] = ticketFareThreshold_test

ticketFareThreshold()

bins = [0,18,45,100]
group_names = ['young','middle','old']
train_new["ageCategories"] = pd.cut(train_new['Age'], bins, labels=group_names)
ageCategoryIntegerConversion = {'young':1, 'middle':2, 'old':3}
train_new['ageGroup'] = train_new['ageCategories'].map(ageCategoryIntegerConversion)
train_new.drop('Age', 1, inplace=True)
train_new.drop('ageCategories', 1, inplace=True)
test_new["ageCategories"] = pd.cut(test_new['Age'], bins, labels=group_names)
test_new['ageGroup'] = test_new['ageCategories'].map(ageCategoryIntegerConversion)
test_new.drop('Age', 1, inplace=True)
test_new.drop('ageCategories', 1, inplace=True)



/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
X_train = train_new.drop("Survived", axis=1)
Y_train = train_new["Survived"]


X_test = test_new
X_train.shape, Y_train.shape, X_test.shape

((891, 12), (891,), (418, 12))

In [3]:
X_train.head()

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,married_female,FamilySize,travelAlonePassenger,bigFamilyPassenger,ticketFareThreshold,ageGroup
0,3,1,0,7.2500,0,1,0,2,0,0,0,2
1,1,1,0,71.2833,1,0,1,2,0,0,1,2
2,3,0,0,7.9250,1,0,0,1,1,0,0,2
3,1,1,0,53.1000,1,0,1,2,0,0,1,2
4,3,0,0,8.0500,0,1,0,1,1,0,0,2


In [4]:
logisticRegressionModel = LogisticRegression()
logisticRegressionModel.fit(X_train,Y_train)
Y_predictions = logisticRegressionModel.predict(X_test)
scoreLogisticRegressionModel = round(logisticRegressionModel.score(X_train,Y_train)*100,2)
scoreLogisticRegressionModel 
pd.DataFrame(list(zip(X_train.columns,np.transpose(logisticRegressionModel.coef_))))

,0,1
0,Pclass,[-0.718503735059]
1,SibSp,[-0.780472013243]
2,Parch,[-0.616876004379]
3,Fare,[0.00374111169057]
4,Sex_female,[2.11422721029]
5,Sex_male,[-0.330967782141]
6,married_female,[0.450694606145]
7,FamilySize,[0.385911410532]
8,travelAlonePassenger,[-0.419027842632]
9,bigFamilyPassenger,[-0.398028399251]


In [5]:
scoreLogisticRegressionModel

80.129999999999995

We'll also try a random forest classifier method and print out a list of the feature importances. The feature importances tells us how important each of our features was in classifying passengers.

In [6]:
#randomForestModel = RandomForestClassifier(n_estimators = 100)
#randomForestModel.fit(X_train,Y_train)
#Y_predictions = randomForestModel.predict(X_test)
#randomForestModel.score(X_train, Y_train)

#features = X_train.columns[:12]
#list(zip(X_train[features], randomForestModel.feature_importances_))

In [7]:
#scoreRandomForestModel = round(randomForestModel.score(X_train,Y_train)*100,2)
#scoreRandomForestModel

In [8]:
kaggleSubmissionFile = pd.DataFrame( { "PassengerId": test_df_predictions["PassengerId"],
                                   "Survived": Y_predictions})
kaggleSubmissionFile.to_csv('submission.csv',index=False)

The above submission receives a score of 0.77511 with a kaggle submission.